In [32]:
import os
import numpy as np
import csv 
import random
import string
from PIL import Image, ImageDraw, ImageFont

In [33]:
output_dir = 'captcha_images/hard'
os.makedirs(output_dir, exist_ok=True)
image_size = (200,80)
font_size = 40
num_samples = 100

# Extended font list with 11 working fonts for more diversity
font_paths = [
    # Original working fonts
    '../fonts/OpenSans-Regular.ttf',
    '../fonts/DejaVuSans.ttf',
    '../fonts/Ubuntu-Regular.ttf',
    # Additional system fonts
    '/usr/share/fonts/liberation/LiberationSans-Regular.ttf',
    '/usr/share/fonts/liberation/LiberationSerif-Regular.ttf', 
    '/usr/share/fonts/liberation/LiberationMono-Regular.ttf',
    '/usr/share/fonts/TTF/DejaVuSerif-Bold.ttf',
    '/usr/share/fonts/TTF/DejaVuSans-Oblique.ttf',
    '/usr/share/fonts/TTF/DejaVuSansCondensed-Bold.ttf',
    '/usr/share/fonts/noto/NotoSans-Regular.ttf',
    '/usr/share/fonts/noto/NotoSerif-Regular.ttf'
]

In [34]:
with open('../ai_wordlist.txt', 'r') as f:
    words = [line.strip() for line in f.readlines()]

In [42]:
def add_noise(image):
    np_img = np.array(image).astype(np.int16)
    noise = np.random.normal(0, 50, np_img.shape) 
    noisy = np.clip(np_img + noise, 0, 255).astype(np.uint8)
    return Image.fromarray(noisy)

In [36]:
def random_caps(word):
    return ''.join(c.upper() if random.random() > 0.5 else c.lower() for c in word)

In [37]:
def get_font(font_paths = font_paths):
    return random.choice(font_paths)

In [43]:
output_file_path = os.path.join(output_dir, 'labels.csv')
i = 0
with open(output_file_path, mode = 'w', newline = '') as file:
    writer = csv.writer(file)
    writer.writerow(['filename', 'label'])

    print(len(words))
    for word in words:
        font_path = get_font()
        word = random_caps(word)
        image = Image.new('RGB', image_size, (255, 255, 255))
        draw = ImageDraw.Draw(image)
        font = ImageFont.truetype(font_path, font_size)

        bbox = draw.textbbox((0, 0), word, font=font)
        text_w, text_h = bbox[2] - bbox[0], bbox[3] - bbox[1]
        position = ((image_size[0]-text_w)//2, (image_size[1]-text_h)//2)
        
        text_color = tuple(random.randint(0, 150) for _ in range(3))
        draw.text(position, word, fill=text_color, font=font)

        image = add_noise(image)

        filename = f"img_{i}.png"
        i+= 1
        image.save(os.path.join(output_dir, filename))
        writer.writerow([filename, word])



100
